In [1]:
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt
import traceback
import time
from datetime import datetime
import cv2

In [2]:
width = 1080
height = 1920
fps = 30

In [3]:
def initialize_cam(cam):
    cam.UserSetSelector = 'Default'
    cam.UserSetLoad.Execute()

    cam.ExposureAuto = 'Off'
    cam.PixelFormat = 'YCbCr422_8'
    cam.ExposureTime = 30000
    cam.AcquisitionFrameRate = fps



In [4]:
tlf = py.TlFactory.GetInstance()
cam = py.InstantCamera(tlf.CreateFirstDevice())
cam.Open()
initialize_cam(cam)

In [5]:
converter = py.ImageFormatConverter()
converter.OutputPixelFormat = py.PixelType_RGB8packed
converter.OutputBitAlignment = "MsbAligned"

In [6]:
def draw_time_date(frame):

    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (1150, 50)
    fontScale = 1
    color = (255, 255, 0)
    thickness = 2

    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

    frame = cv2.putText(frame, dt_string, org, font, fontScale, color, thickness, cv2.LINE_AA)

    # Attempt to display using cv2 (doesn't work)
    cv2.namedWindow("Input")
    cv2.imshow("Input", frame)
    cv2.waitKey(1)
    return frame

In [19]:
output_video = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'MJPG'), fps, (width,height))

def save_video(frame):
    frame = np.frombuffer(frame, dtype=np.uint8).reshape(width, height, -1)
    frame = cv2.cvtColor(cv2.resize(frame, (height,width)), cv2.COLOR_BGR2RGB)

    frame = draw_time_date(frame)
    # output_video.write(frame)


    return

In [15]:
class ImageHandler (py.ImageEventHandler):
    def __init__(self, *args):
        super().__init__(*args)
        self.time_old = time.time()
    
    def OnImageGrabbed(self, camera, grabResult):
        try:
            if grabResult.GrabSucceeded():
                
                if (~converter.ImageHasDestinationFormat(grabResult)):
                    grabResult = converter.Convert(grabResult)
                    
                img = grabResult.Array
                time_new = time.time()
                rate = 1/(time_new-self.time_old)
                print(rate)
                self.time_old = time_new
                # img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB )
                # cv2.namedWindow("Input")
                # cv2.imshow("Input", img)
                # cv2.waitKey(1)
                save_video(img)
            else:
                raise RuntimeError("Grab failed")
        except Exception as e:
            traceback.print_exc()

In [9]:
def BackgroundLoopSample():
    handler = ImageHandler()

    cam.RegisterImageEventHandler(handler, py.RegistrationMode_ReplaceAll, py.Cleanup_None)

    # cam.StartGrabbingMax(100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
    cam.StartGrabbing(py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)

    try:
        while cam.IsGrabbing():
            pass
    except KeyboardInterrupt:
        pass

    cam.StopGrabbing()
    cam.DeregisterImageEventHandler(handler)
    cv2.destroyAllWindows()

    # return handler.img_sum

In [20]:
BackgroundLoopSample()

10.80034711045992
14.964639058658989
24.722111542043404
21.52448399380074
22.2812094940609
19.952828348659203
20.26380655603063
27.828080837568503
29.389782290330945
31.43519677426608
29.167824532854887
24.676295646955694
24.132379001634025
25.704959827420314
33.98123632828324
30.104676868306967
24.701146040682676
29.124481817613688
30.31420703811045
26.386074396542504
30.515125500181885
31.95002932730029
30.690403541506605
29.37413946452458
22.377617602023122
26.387236398409584
30.309168690021966
41.174719729841165
34.86189241305938
28.46432715994924
36.64940058019643
28.36768455581482
33.107611672863044
30.95564379234505
36.418372840149345
37.55812849787329
32.14592610191835
53.03673355841331
28.36231345049803
36.5896136298209
35.539829007685334
22.23290379691815
34.769414417401684
40.899290116233715
25.328381552805908
33.93669493171079
26.67436610044454
32.50566909240272
27.155332262909827
31.082502723412453
27.446220692453164
29.283697549396077
31.916235466001094
26.759115239596028

In [88]:
cam.IsOpen()

True

In [11]:
cam.ResultingFrameRate()

30.00030000300003

In [12]:
cam.Close()